In [1]:
import wrangle as w
import pandas as pd
import numpy as np
import matplotlib as plt


imports loaded successfully, awaiting commands...


## Acquire 

In [2]:
df=w.check_file_exists_gbq('cfpb.csv','service_key.json')

csv file found and loaded


In [3]:
df.head()

,date_received,product,subproduct,issue,subissue,consumer_complaint_narrative,company_public_response,company_name,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,complaint_id
0,2022-08-25,"Credit reporting, credit repair services, or o...",Other personal consumer report,Problem with a company's investigation into an...,Their investigation did not fix an error on yo...,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,CT,06514,NaN,Consent not provided,Web,2022-08-25,Closed with non-monetary relief,True,NaN,5914984
1,2018-03-22,"Money transfer, virtual currency, or money ser...",Mobile or digital wallet,Unauthorized transactions or other transaction...,NaN,On XX/XX/XXXX at XXXX XXXX my sister ( an XXXX...,NaN,"Early Warning Services, LLC",TX,77086.0,Servicemember,Consent provided,Web,2018-03-22,Closed with explanation,True,NaN,2850925
2,2020-03-20,"Payday loan, title loan, or personal loan",Installment loan,Charged fees or interest you didn't expect,NaN,The amount of money going toward the principal...,Company believes it acted appropriately as aut...,"Duvera Billing Services, LLC",PA,19131,NaN,Consent provided,Web,2020-03-20,Closed with explanation,True,NaN,3573868
3,2016-05-17,Credit card,NaN,Identity theft / Fraud / Embezzlement,NaN,NaN,NaN,Resurgent Capital Services L.P.,NaN,NaN,NaN,NaN,Fax,2016-05-27,Closed with explanation,True,False,1929431
4,2016-10-18,Credit card,NaN,Billing disputes,NaN,"I bought XXXX TVs from XXXX. One was fine, the...",NaN,TD BANK US HOLDING COMPANY,CT,06365,Older American,Consent provided,Web,2016-10-18,Closed with monetary relief,True,False,2164764


## Pre cleaning

In [4]:
df.tags.value_counts(dropna=False)

NaN                              3076693
Servicemember                     216614
Older American                    133725
Older American, Servicemember      31874
Name: tags, dtype: int64

In [5]:
df.tags=df.tags.fillna('Normal')

In [6]:
df.tags.value_counts(dropna=False)

Normal                           3076693
Servicemember                     216614
Older American                    133725
Older American, Servicemember      31874
Name: tags, dtype: int64

In [7]:
df.submitted_via.value_counts(dropna=False)

Web             2950186
Referral         239888
Phone            154119
Postal mail       87903
Fax               25659
Web Referral        726
Email               425
Name: submitted_via, dtype: int64

In [8]:
df.consumer_disputed.value_counts(dropna=False)

NaN      2690507
False     620021
True      148378
Name: consumer_disputed, dtype: int64

In [9]:
df.sample(5)

,date_received,product,subproduct,issue,subissue,consumer_complaint_narrative,company_public_response,company_name,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,complaint_id
874676,2020-11-06,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AL,35217.0,Normal,Consent not provided,Web,2020-11-06,Closed with explanation,True,NaN,3940069
1574475,2012-04-01,Mortgage,FHA mortgage,Settlement process and costs,NaN,NaN,NaN,PNC Bank N.A.,VA,23168,Normal,NaN,Web,2012-04-03,Closed without relief,True,False,43168
2185758,2022-10-14,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,In accordance with the Fair Credit Reporting a...,NaN,"EQUIFAX, INC.",TX,77592.0,Normal,Consent provided,Web,2022-10-14,Closed with explanation,True,NaN,6086275
3336082,2022-02-26,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,"I have been in communication with XXXX, XXXX a...",NaN,"EQUIFAX, INC.",GA,30016.0,Normal,Consent provided,Web,2022-02-26,Closed with explanation,True,NaN,5264709
3324780,2021-12-05,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,NaN,Company has responded to the consumer and the ...,SYNCHRONY FINANCIAL,IN,46250.0,Normal,Consent not provided,Web,2021-12-05,Closed with explanation,True,NaN,4977028


In [10]:
df.consumer_complaint_narrative.count()

1246739

## Clean Data:

In [11]:
df=w.clean_data(df)
df

,date_received,narrative,company_name,state,tags,company_response_to_consumer,product_bins
1,2018-03-22,On XX/XX/XXXX at XXXX XXXX my sister ( an XXXX...,"Early Warning Services, LLC",TX,Servicemember,Closed with explanation,money_service
2,2020-03-20,The amount of money going toward the principal...,"Duvera Billing Services, LLC",PA,Normal,Closed with explanation,loans
4,2016-10-18,"I bought XXXX TVs from XXXX. One was fine, the...",TD BANK US HOLDING COMPANY,CT,Older American,Closed with monetary relief,credit_card
7,2023-02-24,I've now missed two closing dates XX/XX/XXXX a...,Home American Mortgage Corp,FL,Normal,Closed with explanation,mortgage
9,2018-02-15,This is a Capitol one prepaid card. The proble...,CAPITAL ONE FINANCIAL CORPORATION,GA,Normal,Closed with explanation,credit_card
...,...,...,...,...,...,...,...
3458884,2021-10-11,In XXXX I consolidated my student loans with F...,FIRST REPUBLIC BANK,MA,Servicemember,Closed with explanation,loans
3458894,2018-07-17,I made a commitment to Chase bank in XX/XX/201...,JPMORGAN CHASE & CO.,TX,Normal,Closed with explanation,bank
3458895,2020-01-15,I am afraid we have had a breach of our origin...,JPMORGAN CHASE & CO.,CA,Normal,Closed with explanation,debt_collection
3458896,2017-10-10,"On XXXX XXXX, XXXX, I faxed a payoff amount re...",M&T BANK CORPORATION,TX,Normal,Closed with explanation,mortgage


In [12]:
df.	company_name.value_counts()

EQUIFAX, INC.                               195447
TRANSUNION INTERMEDIATE HOLDINGS, INC.      172644
Experian Information Solutions Inc.         167388
WELLS FARGO & COMPANY                        32813
BANK OF AMERICA, NATIONAL ASSOCIATION        31978
                                             ...  
Credit Mount                                     1
GREATAMERICA FINANCIAL SERVICES CORP.            1
Colonial Mortgage Service Co. Of America         1
snw investments                                  1
Consumer Defense Group, LLC                      1
Name: company_name, Length: 5597, dtype: int64

In [13]:
boa_df=df[df.company_name=='BANK OF AMERICA, NATIONAL ASSOCIATION']


In [14]:
boa_df.date_received.value_counts()

2018-04-28    46
2022-09-20    36
2022-09-13    34
2023-01-13    34
2022-08-29    34
              ..
2018-11-27     1
2015-12-25     1
2015-09-13     1
2018-07-28     1
2016-05-22     1
Name: date_received, Length: 2898, dtype: int64

## NLTK Language